In [1]:
import os 
import numpy as np 
import matplotlib.pyplot as plt

def GetIQ(root_dir):
    Threshold = 0.0025
    Fsize = 1850
    data4 = np.fromfile(root_dir, count= 731060520, dtype=np.complex64)
    I4 = np.real(data4)
    Q4 = np.imag(data4)
    Frames = []
    i = 1
    start_indices = np.where(np.abs(I4) >= Threshold)[0]
    for index in start_indices:
        if len(I4[index: index+Fsize])==Fsize and len(Q4[index: index+Fsize])==Fsize and np.all(np.abs(data4[index-10000:index])<4*Threshold) and not np.all(np.abs(data4[index:index+5])<7*Threshold):
            #print("#", i, "The index",index, "is",len(I4[index: index+Fsize])==Fsize and len(Q4[index: index+Fsize])==Fsize and np.all(np.abs(I4[index-178150:index])<5*Threshold))

            i += 1

            Frame = np.dstack((I4[index:index+Fsize], Q4[index:index+Fsize]))
            Frames.append(Frame)
    print("number of Frames:", i)
    return np.vstack(Frames)

def GetPWR(signal):
    Pwr = np.sum(signal**2)
    return Pwr

def Ifilter(Frames, Ref_Pwr):
    Del_inx = []
    d = 0
    for i in range(Frames.shape[0]):
        tmp_pwr = GetPWR(Frames[i,:,0])
        if tmp_pwr <= Ref_Pwr:
            Del_inx.append(i)
            d += 1
            #print("detect index", i)

    CFrames = np.delete(Frames, Del_inx, axis=0)
    print("Detect ", d, "Bad I Frames :(")
    return CFrames

def Dfilter(Frames):
    N = Frames.shape[0]
    Db_inx = []
    for i in range(N):
        if i != (N-1):
            if round(GetPWR(Frames[i,:,0]),3) == round(GetPWR(Frames[i+1,:,0]),3):
                Db_inx.append(i)
    NDFrames = np.delete(Frames, Db_inx, axis=0)
    print("Detect ", len(Db_inx), "Bad I Frames :(")
    return NDFrames

def GetLP(Frames, num):
    temp = 0
    Lowest = [] 
    for i in range(Frames.shape[0]):
        if i == 0:
            temp = GetPWR(Frames[i,:,0])

        p = GetPWR(Frames[i,:,0])
        if p < temp:
            temp = p 
            Lowest.append(temp)
            #plt.plot(Frames[i,:,0])
            #plt.show()
            #print(p)
            
    Lowst = np.array(Lowest)
    l50 = np.sort(Lowst)[:num]
    print(f"The lowest power is {temp}\nAnd the lowest 50 are {l50}")
    print("===========================================================")
    return np.max(l50)

def Qfilter(Frames, Ref_Pwr):
    Lowest = Ref_Pwr
    Highest = 0
    Bad_inx = []
    d =0 
    for i in range(Frames.shape[0]):
        p = GetPWR(Frames[i,:,1])
        if p < Lowest:
            #print("Ops")
            Bad_inx.append(i)
            d +=1 
        if p > Highest:
            Highest = p
            #x = i
    CFrames = np.delete(Frames, Bad_inx, axis=0)
    print("Detect ", d, "Bad Q Frames :(")
    return CFrames

In [ ]:
I_dir = "...."
O_dir = "...."

for class_dir in (os.listdir(I_dir)):
    Input_dir = os.path.join(I_dir, class_dir)
    out_dir = os.path.join(O_dir, class_dir)
    class_dir = Input_dir
    print("Reading From ==>")
    print(class_dir)
    x1 = GetIQ(class_dir)
    Th = GetLP(x1, 50)
    x1 = Dfilter(x1)
    x2 = Ifilter(x1,10)
    x3 = Qfilter(x2, 10)
    x3 = np.swapaxes(x3, 1, 2)
    print("Saving to ==>")
    print(out_dir)
    np.save(out_dir, x3)
    print("=========================================================================================")
    print("=========================================================================================")